PC 환경

In [ ]:
# @title 라이브러리 사용 정리

import time
import json
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2


In [ ]:
# @title 로컬 경로 설정 및 JSON 파일 경로 수집

# Mac 로컬 환경의 데이터 기본 경로 설정
base_data_dir = '/Users/bellboi/data/ai05-level1-project'
train_annotations_dir = os.path.join(base_data_dir, 'train_annotations') # train_annotations 폴더 경로

json_file_paths = [] # 모든 JSON 파일의 경로를 저장할 리스트

print(f"로컬 데이터 경로 설정 완료: {base_data_dir}")

start_time = time.time()

if os.path.exists(train_annotations_dir):
    for root, dirs, files in os.walk(train_annotations_dir):
        for file_name in files:
            if file_name.endswith('.json'):
                full_path = os.path.join(root, file_name)
                json_file_paths.append(full_path)

    end_time = time.time()

    print(f"'train_annotations'에서 발견된 총 JSON 파일 수: {len(json_file_paths)}개")
    print(f"경로 수집 완료 시간: {end_time - start_time:.2f}초")
    print(f"첫 번째 JSON 파일 경로 (예시): {json_file_paths[0]}")

else:
    print(f"로컬 경로를 찾을 수 없습니다. {train_annotations_dir}")

In [ ]:
# @title JSON 파일 파싱 및 오류 카운트

total_files = len(json_file_paths)

parsed_data = []
error_files = []
success_count = 0
error_count = 0

print(f" 총 {total_files}개의 JSON 파일을 파싱합니다.")

for file_path in json_file_paths:
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        image_info = data['images'][0]
        file_name = image_info['file_name']
        image_width = image_info['width']
        image_height = image_info['height']

        annotations = data.get('annotations', [])

        for ann in annotations:
            bbox = ann['bbox']
            category_id = ann['category_id']

            parsed_data.append({
                'image_file_name': file_name,
                'category_id': category_id,
                'bbox_x': bbox[0],
                'bbox_y': bbox[1],
                'bbox_w': bbox[2],
                'bbox_h': bbox[3],
                'image_width': image_width,
                'image_height': image_height
            })

        success_count += 1

    except Exception as e:
        error_count += 1
        error_files.append(file_path)
        pass

df = pd.DataFrame(parsed_data)

print("\n 데이터 파싱 완료 및 DataFrame 정보 ")
print(f"총 {len(df)}개의 알약 객체(Row)를 추출했습니다. (데이터 로우 수)")

print(f" 총 시도한 파일 수: {total_files}개")
print(f" 성공적으로 파싱된 파일 수: {success_count}개")
print(f" **파싱에 실패한 파일 수:** **{error_count}개**")
print(f"데이터 손실률: {error_count / total_files * 100:.2f}%")

구글 클라우드와 파일을 연동하거나 icloud를 연동하면 데이터 손실이 무조건 나옵니다

In [ ]:
# @title 바운딩 박스 이상치 필터링 및 이상치 이미지 시각화

base_image_dir = '/Users/bellboi/data/ai05-level1-project/train_images'

df_temp = df.copy()

# 너비높이 0 보다 작은수 필터링
df_filtered_temp = df_temp[
    (df_temp['bbox_w'] > 0) &
    (df_temp['bbox_h'] > 0)
].copy()

# 바운딩박스 x y축 계산
df_filtered_temp['bbox_x_max'] = df_filtered_temp['bbox_x'] + df_temp['bbox_w']
df_filtered_temp['bbox_y_max'] = df_filtered_temp['bbox_y'] + df_temp['bbox_h']

# 이미지 경계를 벗어나는 데이터 필터링
df_filtered_temp = df_filtered_temp[
    (df_filtered_temp['bbox_x_max'] <= df_filtered_temp['image_width']) &
    (df_filtered_temp['bbox_y_max'] <= df_filtered_temp['image_height'])
]

removed_indices = df_temp.index.difference(df_filtered_temp.index)
df_outliers = df_temp.loc[removed_indices].copy()
outlier_files = df_outliers['image_file_name'].unique().tolist()

df_original_len = len(df)

print(f" 원본갯수 : {df_original_len}")

num_outlier_files = len(outlier_files)
print(f"바운딩 박스가 이상한 json 파일: {num_outlier_files} 개")

MAX_VISUALIZE = 5

for i, file_name in enumerate(outlier_files):
    if i >= MAX_VISUALIZE:
        break

    image_path = os.path.join(base_image_dir, file_name)

    if not os.path.exists(image_path):
        print(f"Warning: Image file not found at path: {image_path}")
        continue

    image = cv2.imread(image_path)

    if image is None:
        print(f"Warning: Could not load image file: {image_path}")
        continue

    outlier_bboxes = df_outliers[df_outliers['image_file_name'] == file_name]

    print(f"\n[File: {file_name} | Image Size: {image.shape[1]}x{image.shape[0]}]")

    for index , row in outlier_bboxes.iterrows():
        x, y, w, h = int(row['bbox_x']), int(row['bbox_y']), int(row['bbox_w']), int(row['bbox_h'])
        print(f"  - Outlier {index}: x={x}, y={y}, w={w}, h={h}")
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(8, 8))
    plt.imshow(image_rgb)
    plt.title(f"Outlier BBox Visualization: {file_name}", fontsize=12)
    plt.axis('off')
    plt.show()


In [ ]:
# @title 이상치 X좌표 수정 및 재검증 시각화


outlier_files = df_outliers['image_file_name'].unique().tolist()
MAX_VISUALIZE = 5

for i, file_name in enumerate(outlier_files):
    if i >= MAX_VISUALIZE:
        break

    image_path = os.path.join(base_image_dir, file_name)
    if not os.path.exists(image_path):
        print(f"이미지 패스를 못찾았을때: {image_path}")
        continue

    image = cv2.imread(image_path)
    if image is None:
        print(f"이미지를 로드하지 못했을때: {image_path}")
        continue

    outlier_bboxes = df_outliers[df_outliers['image_file_name'] == file_name].copy()

    is_6567_outlier = outlier_bboxes['bbox_x'] == 6567

    if is_6567_outlier.any():
        outlier_bboxes.loc[is_6567_outlier, 'bbox_x'] = 567
        print(f"\n[File: {file_name}] 데이터 수정 4자리에서 뒷자리한자리 제거")

    is_8889_outlier = outlier_bboxes['bbox_y'] == 8889

    if is_8889_outlier.any():
        outlier_bboxes.loc[is_8889_outlier, 'bbox_y'] = 888
        print(f"\n[File: {file_name} | Image Size: {image.shape[1]}x{image.shape[0]}]")
        modified = True
    else:
        print(f"\n[File: {file_name} | Image Size: {image.shape[1]}x{image.shape[0]}]")

    for index, row in outlier_bboxes.iterrows():
        x, y, w, h = int(row['bbox_x']), int(row['bbox_y']), int(row['bbox_w']), int(row['bbox_h'])

        print(f"  - BBox {index}: x={x}, y={y}, w={w}, h={h}")

        cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(8, 8))
    plt.imshow(image_rgb)
    plt.title(f"MODIFIED BBox Visualization: {file_name} (x=6567 -> x=656)", fontsize=12)
    plt.axis('off')
    plt.show()

In [ ]:
# @title 바운딩 박스 중첩 분석 및 시각화

base_data_dir = '/data/ai05-level1-project'
image_dir = base_image_dir
json_dir = json_file_paths

df['bbox_x_max'] = df['bbox_x'] + df['bbox_w']
df['bbox_y_max'] = df['bbox_y'] + df['bbox_h']

def calculate_iou(box_a, box_b):

    x_a = max(box_a['bbox_x'], box_b['bbox_x'])
    y_a = max(box_a['bbox_y'], box_b['bbox_y'])
    x_b = min(box_a['bbox_x_max'], box_b['bbox_x_max'])
    y_b = min(box_a['bbox_y_max'], box_b['bbox_y_max'])

    inter_w = max(0, x_b - x_a)
    inter_h = max(0, y_b - y_a)
    inter_area = inter_w * inter_h

    area_a = box_a['bbox_w'] * box_a['bbox_h']
    area_b = box_b['bbox_w'] * box_b['bbox_h']
    union_area = area_a + area_b - inter_area

    if union_area == 0:
        return 0.0

    return inter_area / union_area


# 중첩되는 바운딩 박스 쌍을 저장할 리스트입니다.
overlapping_bboxes = []
# IoU 임계값 설정 절대 겹치는 일은물리적으로 불가능하니 0.1로설정
IOU_THRESHOLD = 0.01

for file_name, group in df.groupby('image_file_name'):

    for i in range(len(group)):
        for j in range(i + 1, len(group)):

            box_a = group.iloc[i]
            box_b = group.iloc[j]

            # IoU 수치 계산
            iou = calculate_iou(box_a, box_b)

            if iou >= IOU_THRESHOLD:
                overlapping_bboxes.append({
                    'image_file_name': file_name,
                    'iou': iou,
                    'box_a_id': box_a.name,
                    'box_b_id': box_b.name,
                    'category_a': box_a['category_id'],
                    'category_b': box_b['category_id']
                })

df = df.drop(columns=['bbox_x_max', 'bbox_y_max'])

print("--- Bounding Box Overlap Analysis Summary ---")
print(f"Total overlapping pairs found (IoU >= {IOU_THRESHOLD * 100:.0f}%): {len(overlapping_bboxes)} pairs")
print(f"The list 'overlapping_bboxes' now contains the detailed information of these {len(overlapping_bboxes)} pairs.")

df_overlapping = pd.DataFrame(overlapping_bboxes)
# IoU가 높은 순서대로 정렬하여 상위 5개를 추출합니다.
df_head = df_overlapping.sort_values(by='iou', ascending=False)

print("\nTop 5 Overlapping Pairs (First 5 items in the list):")
# to_string()을 사용하여 라이브러리 설치 없이 표 형식으로 출력합니다.
print(df_head.to_string(index=False))


In [ ]:
# @title 중첩 바운딩 박스 시각화

# 중첩되는 파일을 저장할 딕셔너리 (시각화 목적)
overlapping_files_to_visualize = {}

for item in overlapping_bboxes:

    file_name = item['image_file_name']
    box_a_data = df.loc[item['box_a_id']]
    box_b_data = df.loc[item['box_b_id']]

    if file_name not in overlapping_files_to_visualize:

        group_df_for_file = df[df['image_file_name'] == file_name].copy()
        overlapping_files_to_visualize[file_name] = {
            'image_path': os.path.join(image_dir, file_name),
            'iou_pairs': [],
            'image_df_group': group_df_for_file
        }

    overlapping_files_to_visualize[file_name]['iou_pairs'].append({
        'iou': item['iou'],

        'box_a_id': item['box_a_id'],
        'box_b_id': item['box_b_id'],

        'bbox_a_cords': (box_a_data['bbox_x'], box_a_data['bbox_y'], box_a_data['bbox_w'], box_a_data['bbox_h']),
        'category_a': item['category_a'],
        'bbox_b_cords': (box_b_data['bbox_x'], box_b_data['bbox_y'], box_b_data['bbox_w'], box_b_data['bbox_h']),
        'category_b': item['category_b']
    })


MAX_VISUALIZE = 20

for i, (file_name, data) in enumerate(overlapping_files_to_visualize.items()):
    if i >= MAX_VISUALIZE:
        break

    image_path = data['image_path']
    group_df = data['image_df_group']

    if not os.path.exists(image_path):
        print(f"파일없음 {image_path}")
        continue

    image = cv2.imread(image_path)

    if image is None:
        print(f"이미지 로드 실패 {image_path}")
        continue

    print(f"--- Visualizing Overlaps in: {file_name} ---")

    for pair in data['iou_pairs']:

        x_a, y_a, w_a, h_a = [int(x) for x in pair['bbox_a_cords']]

        cv2.rectangle(image, (x_a, y_a), (x_a + w_a, y_a + h_a), (0, 0, 255), 2)
        cv2.putText(image, f"ID: {pair['box_a_id']}", (x_a, y_a - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)

        x_b, y_b, w_b, h_b = [int(x) for x in pair['bbox_b_cords']]

        cv2.rectangle(image, (x_b, y_b), (x_b + w_b, y_b + h_b), (0, 0, 255), 2)
        cv2.putText(image, f"ID: {pair['box_b_id']}", (x_b, y_b + h_b + 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)

        print(f"  - IoU: {pair['iou']:.2f} | BBox A ID: {pair['box_a_id']} (Cat: {pair['category_a']}) | BBox B ID: {pair['box_b_id']} (Cat: {pair['category_b']})")

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(10, 10))
    plt.imshow(image_rgb)
    plt.title(f"BBox: {file_name} (Pairs: {len(data['iou_pairs'])})", fontsize=12)
    plt.axis('off')
    plt.show()

눈으로 확인해봤을때
K-003351-033880-035206_0_2_0_2_75_000_200.png
이파일을 제외하고 나머지파일을 변환하기로했습니다

In [ ]:
filtered_overlapping_bboxes = [
    item for item in overlapping_bboxes
    if item['image_file_name'] != 'K-003351-033880-035206_0_2_0_2_75_000_200.png'
]

In [ ]:
# @title JSON 경로 찾기 헬퍼 함수 (이전 코드와 동일한 로직 사용)
def find_json_path_by_image_name(image_name, all_json_paths):
    json_prefix = image_name.split('_')[0]
    for path in all_json_paths:
        if json_prefix in os.path.basename(path) and path.endswith('.json'):
            return path
    return None

In [ ]:
def visualize_and_show_multiple_json(overlap_list, df_all, image_base_dir, all_json_paths):
    """
    모든 중첩된 바운딩 박스 쌍을 시각화하고, A/B Annotation이 추출된 JSON 파일 내용을 각각 출력합니다.
    """

    df_overlapping = pd.DataFrame(overlap_list).sort_values(by='iou', ascending=False).reset_index(drop=True)
    total_pairs = len(df_overlapping)

    print(f"--- 총 {total_pairs}쌍의 중첩 바운딩 박스 정보를 시각화하고 관련 JSON 파일들을 출력합니다. ---")

    for i, item in df_overlapping.iterrows():
        file_name = item['image_file_name']
        box_a_id = item['box_a_id']
        box_b_id = item['box_b_id']
        iou_value = item['iou']

        print(f"\n==========================================================================================")
        print(f"### [중첩 쌍 {i+1}/{total_pairs}] 파일: {file_name} | IoU: {iou_value:.4f}")

        # 중첩된 두 박스 데이터만 추출
        box_a = df_all.loc[box_a_id]
        box_b = df_all.loc[box_b_id]

        # 1. 시각화: BBox 2개 모두 그리기 (선 굵기 4, 빨간색 고정)
        image_path = os.path.join(image_base_dir, file_name)
        image = cv2.imread(image_path)

        if image is None:
             print(f"경고: 이미지 로드 실패 또는 파일 없음: {image_path}")
             continue

        # 바운딩 박스 색상 고정 (요청 반영)
        COLOR_FIXED = (0, 0, 255)  # BGR: 빨간색
        THICKNESS = 4
        FONT_SCALE = 0.7
        TAG_BACKGROUND = (255, 255, 255)
        TAG_COLOR = (0, 0, 0)

        def draw_bbox_and_tag(img, row, tag_text, color):
            # 좌표는 이미 int로 변환되어 저장되어 있으나, 안전을 위해 다시 int로 변환
            x, y, w, h = int(row['bbox_x']), int(row['bbox_y']), int(row['bbox_w']), int(row['bbox_h'])
            cv2.rectangle(img, (x, y), (x + w, y + h), color, THICKNESS) # 바운딩 박스 그리기

            # 태그 배경 박스 그리기
            (text_w, text_h), baseline = cv2.getTextSize(tag_text, cv2.FONT_HERSHEY_SIMPLEX, FONT_SCALE, 2)
            cv2.rectangle(img, (x, y - text_h - 10), (x + text_w + 5, y), TAG_BACKGROUND, -1)

            # 태그 텍스트 그리기
            cv2.putText(img, tag_text, (x, y - baseline - 3),
                        cv2.FONT_HERSHEY_SIMPLEX, FONT_SCALE, TAG_COLOR, 2, cv2.LINE_AA)

        # Box A 그리기
        tag_a = f"A: ID {box_a_id} | Cat {box_a['category_id']}"
        draw_bbox_and_tag(image, box_a, tag_a, COLOR_FIXED)

        # Box B 그리기 (A와 같은 빨간색으로, 두 개의 BBox가 겹치더라도 모두 그려짐)
        tag_b = f"B: ID {box_b_id} | Cat {box_b['category_id']}"
        draw_bbox_and_tag(image, box_b, tag_b, COLOR_FIXED) # 두 번째 BBox 그리기

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(10, 10))
        plt.imshow(image_rgb)
        plt.title(f"Overlapping Pair {i+1}: {file_name} (IoU: {iou_value:.4f}, Both in RED)", fontsize=12)
        plt.axis('off')
        plt.show()

        # 2. JSON 내용 출력: A/B 각각의 JSON 파일 전체 내용을 출력 (요청 반영)

        # Box A에 해당하는 JSON 파일 경로 찾기
        json_path_a = find_json_path_by_image_name(file_name, all_json_paths)

        # Box B에 해당하는 JSON 파일 경로 찾기 (이 경우 A와 동일할 확률이 높지만, 경로 탐색 로직은 동일)
        json_path_b = find_json_path_by_image_name(file_name, all_json_paths)

        # JSON 출력 헬퍼 함수
        def print_full_json(json_path, label):
            if json_path:
                print(f"\n  >> {label} 전체 JSON File Content (경로: {json_path}):")
                try:
                    with open(json_path, 'r', encoding='utf-8') as f:
                        json_data = json.load(f)
                    print(json.dumps(json_data, indent=4, ensure_ascii=False))
                except Exception as e:
                    print(f"JSON 파일 ({json_path})을 읽는 중 오류 발생: {e}")
            else:
                 print(f"{label} JSON 파일 경로를 찾을 수 없습니다.")

        # Box A JSON 출력
        print_full_json(json_path_a, f"Annotation A (BBox ID: {box_a_id})")

        # Box B JSON 출력 (A와 경로가 같더라도, 사용자 요청에 따라 내용을 다시 출력)
        # 단, 출력 시 혼동을 줄이기 위해 경로가 다를 때만 '다른 파일'로 간주하고,
        # 경로가 같으면 '같은 파일이지만 요청에 따라 다시 출력'한다고 명시합니다.

        if json_path_a != json_path_b:
            # 경로가 다를 때
            print_full_json(json_path_b, f"Annotation B (BBox ID: {box_b_id})")
        else:
            # 경로가 같을 때
            print_full_json(json_path_b, f"Annotation B (BBox ID: {box_b_id}) - A와 동일 파일")


In [ ]:
# 함수 실행
# json_file_paths 변수가 이전 코드 셀에 의해 정의되어 있어야 합니다.
visualize_and_show_multiple_json(
    overlap_list=filtered_overlapping_bboxes,
    df_all=df,
    image_base_dir=base_image_dir,
    all_json_paths=json_file_paths
)